In [1]:
import pandas as pd
import re

FILE = 'salix008_tab.txt'

with open(FILE,'r') as f:
    out = f.read()
    out = re.sub("(  +)", ",", out)
    out = re.sub("\*\*\*\*\*\*\*", "", out)
    with open(f"{FILE}_processed",'w') as o:
        o.write(out)

salix_df = pd.read_csv(f"{FILE}_processed")
wheatfen_df = pd.read_excel("/home/markhaoxiang/Projects/salix/Wheatfen_treecores.xlsx")

wheatfen_df_diameter = wheatfen_df[['ID', 'Diameter/cm']].dropna()
wheatfen_df_diameter['ID'] = wheatfen_df_diameter['ID'].apply(lambda x : x.replace("?",""))

FileNotFoundError: [Errno 2] No such file or directory: '/home/markhaoxiang/Projects/salix/Wheatfen_treecores.xlsx'

In [ ]:
salix_df

In [ ]:
salix_diameter_df = salix_df.get(["year"])
for wfp in salix_df.columns[0:]:
    d = wheatfen_df_diameter[(wheatfen_df_diameter['ID']==wfp[:-1])]['Diameter/cm'].values
    # Data not available
    if len(d) != 1:
        continue
    d = d[0]
    results = [d]
    for year in range(2022,1986,-1):
        x = salix_df[(salix_df['year']==year)][wfp].values[0]
        d = d - x*2/10
        results.append(round(d,5))
    results.reverse()
    salix_diameter_df[f"{wfp}_diameter"] = pd.Series(results)
     

In [ ]:
salix_diameter_df    

In [ ]:
# define parameters for different species
biomass_params = {"Alnus": (0.0859, 2.3537), "Acer": (-2.7606, 2.5189),"Quercus": (-2.3223,2.4029)}


# for salix biomass calculation: 
# a1=2.4721
# b1=2.4987
# a2=4.5086
# b2=1.9234
# b3=0.2613
#branches_biomass = a1 * dbh ** b1
# stem_biomass = a2 * (dbh ** b2) * (dbh**b3)

salix_basal_area_df = salix_df.get(["year"])
# calculate basal area, biomass and carbon
for ID_diameter in salix_df.columns[0:]:
    basal_area=[]
    biomass=[]
    carbon=[]
    for year in range(1986,2023,1):
        dbh = salix_df[(salix_df['year']==year)][ID_diameter].values[0]
        ba = (np.pi / 4) * dbh ** 2
        basal_area.append(ba)
#         a, b = params[Alnus]
        a = 0.0859
        b = 2.3537
        bio = a * dbh ** b
        car = bio*0.47
        biomass.append(bio)
        carbon.append(car)
    salix_df[f"{ID_diameter}_basal_area"] = pd.Series(basal_area)
    salix_df[f"{ID_diameter}_biomass"] = pd.Series(biomass)
    salix_df[f"{ID_diameter}_carbon"] = pd.Series(carbon)

salix_df

# # calculate increment
bai=[]
bioi=[]
cari=[]